### Graph-based SIR
To generate the node atomic models, up to a degree of N (for example, 20), run the following command from `random_graphs/nodes_atomic_generator`:

```python model_generator.py -o ../user_models -c ../../cdpp/src -m 20 && make -C ../user_models/```

This will generate the atomic sources for each kind of node, and compile CD++ with them.

In [1]:
from pringles.simulator import Simulator
from pringles.models import Coupled
from pringles.utils import VirtualTime
from pringles.display import ModelDisplay

In [2]:
simulator = Simulator(cdpp_bin_path="user_models/bin", user_models_dir="user_models/")

In [3]:
r = simulator.get_registry()

In [24]:
def build_complete_graph_of_nodes(n, exp_lambda, infection_rate):
    nodes = [r.Node1("node%d" % (i), exp_lambda=exp_lambda, infection_prob=infection_rate) for i in range(n)]
    graph = Coupled("top", nodes)
    for i in range(0,n):
        # Connecting i-th node with all others
        for j in range(0,n):
            if i != j:
                graph.add_coupling(nodes[i].get_port("out0"), nodes[j].get_port("in0"))
    return graph

In [27]:
top = build_complete_graph_of_nodes(5, 1, .5)

In [28]:
ModelDisplay(top)

In [29]:
results = simulator.run_simulation(top, VirtualTime.of_seconds(30))

In [30]:
print(results.logs_dfs.keys())

dict_keys(['node4', 'node3', 'node2', 'node1', 'node0', 'top', 'ParallelRoot'])


In [31]:
print("Ran command: " + " ".join(results.process_result.args) + "\n")
print(results.get_process_output())

Ran command: user_models/bin/cd++ -m/var/folders/kk/51hd52l93nz63xzxf2gd0k080000gp/T/tmpr80sy2y5/top_model -LXY -t00:00:30:000 -l/var/folders/kk/51hd52l93nz63xzxf2gd0k080000gp/T/tmpr80sy2y5/logs -o/var/folders/kk/51hd52l93nz63xzxf2gd0k080000gp/T/tmpr80sy2y5/output

PCD++: A Tool to Implement n-Dimensional Cell-DEVS models
Version 3.0 - March 2003
Troccoli A., Rodriguez D., Wainer G., Barylko A., Beyoglonian J., Lopez A.
-----------------------------------------------------------------------------
PCD++ Extended States: An extended and improved version of CD++ for Cell-DEVS
Version 4.1.2 - December 2018
Santi L., Castro, R., Pimás, J.
-----------------------------------------------------------------------------
Discrete Event Simulation Lab
Departamento de Computación
Facultad de Ciencias Exactas y Naturales
Universidad de Buenos Aires, Argentina
-----------------------------------------------------------------------------
Compiled for standalone simulation


Loading models from /var/fo

In [34]:
results.logs_dfs["node1"]

,0,1,message_type,time,model_origin,port,value,model_dest
0,0,L,X,00:00:01:051,top(06),in0,1.0,node1(02)
1,0,L,X,00:00:02:453,top(06),in0,1.0,node1(02)
2,0,L,X,00:00:03:122,top(06),in0,1.0,node1(02)
3,0,L,X,00:00:05:677,top(06),in0,1.0,node1(02)
4,0,L,X,00:00:08:572,top(06),in0,1.0,node1(02)
